# Scraping

#### Look at the Links

In [ ]:
from bs4 import BeautifulSoup # the library that provides the HTTP calls etc.
from urllib.request import urlopen


url = "https://ai.google/research/pubs/?year=2019" # The url we want to scrape
raw = urlopen(url).read() # open the page and get the html
soup = BeautifulSoup(raw ,"html.parser") # parse html
links = [l.get("href") for l in soup.find_all("a")] # identify the links
links = [l for l in links if (l is not None)] # filter the empty links


links

#### Just to check: Getting a page, parsing the html code, looking at the html

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

html = urllib.request.urlopen("https://research.google/pubs/pub47724/")

raw = html.read() 
soup = BeautifulSoup(raw) 
print(raw)

### Scraping a list of pages

#### Creating a list of links

In [ ]:
j=47700 
paperLinks = []

for i in range(10):
    paperLinks.append("https://ai.google/research/pubs/pub" + str(j))
    j=j+1
    
    
paperLinks

#### Get the titles and abstracts

In [ ]:
from bs4 import BeautifulSoup
import urllib.request

title = list()
abstract = list()

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

for l in paperLinks:
    try:
        html = urllib.request.urlopen(l)
    except Exception as e:
        print(e)
    else:
        for l in paperLinks:
            raw = html.read() 
            soup = BeautifulSoup(raw) 
            try:
               for t in soup.find_all("meta"):
                    if t.get("name", None) == "citation_title":
                        title.append(t.get("content", None))
            except Exception as e:
                print(e)
                continue
            try:
                abstract = abstract + [a.string.strip() for a in soup.find_all("p")]
            except Exception as e:
                print(e)
                continue

In [ ]:
title

In [ ]:
abstract

In [ ]:
papers = list(zip(title, abstract))
print(papers)

### Analysing the content

In [ ]:
words = [ w.lower() for t in abstract for w in t.split() ]
words[:20]

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
stop = nltk.corpus.stopwords.words('english')
words_without_stopwords = [i for i in words if i not in stop]
print (words_without_stopwords[:10])

In [ ]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()


raw = " ".join(abstract)
tokens = nltk.word_tokenize(raw)
text = nltk.Text(tokens)

finder = BigramCollocationFinder.from_words(text)
finder.nbest(bigram_measures.pmi, 10)

In [ ]:
text.concordance('learning')

In [ ]:
print(text.similar("effective"))